In [309]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [310]:
df_train_tran = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')


In [311]:
df_train_id = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')


In [312]:
df_test_tran = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')


In [313]:
df_test_id = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')


In [314]:
df_train = pd.merge(df_train_tran, df_train_id, on='TransactionID', how='left')
df_test = pd.merge(df_test_tran, df_test_id, on='TransactionID', how='left')


In [315]:
df_train.shape

(590540, 434)

In [316]:
X=df_train.drop(columns=['isFraud'])
y=df_train['isFraud']

In [317]:
X.shape

(590540, 433)

In [318]:
print(y.value_counts())

isFraud
0    569877
1     20663
Name: count, dtype: int64


In [319]:
from sklearn.model_selection import train_test_split
#same ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)


In [320]:
print(y_test.value_counts())

isFraud
0    113975
1      4133
Name: count, dtype: int64


In [321]:
X_train.shape

(472432, 433)

In [323]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
#if null values>= 80percent -> drop
class DropNulls(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.columns_to_drop_ = None

    def fit(self, X, y=None):
        null_frac = X.isnull().mean()
        self.columns_to_drop_ = null_frac[null_frac > self.threshold].index.tolist()
        return self

    def transform(self, X):
        return X.drop(columns=self.columns_to_drop_, errors='ignore')

#correlated numvalues -> drop
class DropCorr(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.9):
        self.threshold = threshold
        self.columns_to_drop_ = None

    def fit(self, X, y=None):
        X_num = X.select_dtypes(include=[np.number])
        corr_matrix = X_num.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        self.columns_to_drop_ = [column for column in upper.columns if any(upper[column] > self.threshold)]
        return self

    def transform(self, X):
        return X.drop(columns=self.columns_to_drop_, errors='ignore')


#cat handling
class SplitCatCols(BaseEstimator, TransformerMixin):
    def __init__(self, max_unique_for_onehot=3):
        self.max_unique_for_onehot = max_unique_for_onehot
        self.low_cardinality_cols_ = []
        self.high_cardinality_cols_ = []

    def fit(self, X, y=None):
        cat_cols = X.select_dtypes(include=['object', 'category']).columns
        for col in cat_cols:
            n_unique = X[col].nunique(dropna=False)
            if n_unique <= self.max_unique_for_onehot:
                self.low_cardinality_cols_.append(col)
            else:
                self.high_cardinality_cols_.append(col)
        return self

    def transform(self, X):
        return X  


In [324]:
def droptoomanynulls(threshold=0.8):
    return DropNulls(threshold=threshold)


In [40]:
from sklearn.compose import make_column_selector 

def woeandcathandling(max_unique_for_onehot=3):
    numerical_transformer = SimpleImputer(strategy='median')
    onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    woe_encoder = ce.WOEEncoder()

    splitter = SplitCatCols(max_unique_for_onehot=max_unique_for_onehot)
    return Pipeline(steps=[
        ('split_cat', splitter),
        ('encode', ColumnTransformer(transformers=[
            ('num', numerical_transformer, make_column_selector(dtype_include=['int64', 'float64'])),
            ('onehot', onehot_encoder, make_column_selector(dtype_include=['object', 'category'])),
            ('woe', woe_encoder, make_column_selector(dtype_include=['object', 'category']))  
        ], remainder='drop'))
    ])

In [41]:
def corelation(threshold=0.9):
    return DropCorr(threshold=threshold)


In [48]:


model_pipe = Pipeline(steps=[
    ('droptoomanynulls', droptoomanynulls(threshold=0.8)),
    ('corelation', corelation(threshold=0.9)), 
    ('woeandcathandling', woeandcathandling(max_unique_for_onehot=3)),
    ('classifier', LogisticRegression(max_iter=1000))
])


In [43]:
from sklearn.preprocessing import StandardScaler

model_pipeline = Pipeline(steps=[
    ('droptoomanynulls', droptoomanynulls(threshold=0.8)),
    ('corelation', corelation(threshold=0.9)), 
    ('woeandcathandling', woeandcathandling(max_unique_for_onehot=3)),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])


In [44]:
%pip install mlflow
%pip install dagshub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
!pip install dagshub --upgrade


In [45]:
import dagshub
dagshub.init(repo_owner='electrolizzys', repo_name='Fraud_Detection', mlflow=True)


Initialized MLflow to track repo "electrolizzys/Fraud_Detection"

Repository electrolizzys/Fraud_Detection initialized!

In [46]:
with mlflow.start_run(run_name="logistic_run"):
    
    model_pipeline.fit(X_train, y_train)
    
    y_pred_proba = model_pipeline.predict_proba(X_test)[:, 1]
    
    from sklearn.metrics import roc_auc_score
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    mlflow.log_metric("roc_auc", roc_auc)
    
    mlflow.sklearn.log_model(model_pipeline, artifact_path="model")

    print(f"Logged model with ROC AUC: {roc_auc:.4f}")


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
2025/04/27 08:08:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged model with ROC AUC: 0.8597
🏃 View run logistic_run at: https://dagshub.com/electrolizzys/Fraud_Detection.mlflow/#/experiments/0/runs/b41e5d537375412dafd4e41a5ada750f
🧪 View experiment at: https://dagshub.com/electrolizzys/Fraud_Detection.mlflow/#/experiments/0


In [50]:
with mlflow.start_run(run_name="log_wo_scaler"):
    
    model_pipe.fit(X_train, y_train)
    
    y_pred_proba = model_pipe.predict_proba(X_test)[:, 1]
    
    from sklearn.metrics import roc_auc_score
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    mlflow.log_metric("roc_auc", roc_auc)
    
    mlflow.sklearn.log_model(model_pipe, artifact_path="model")

    print(f"Logged model with ROC AUC: {roc_auc:.4f}")


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
2025/04/27 08:32:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged model with ROC AUC: 0.6767
🏃 View run log_wo_scaler at: https://dagshub.com/electrolizzys/Fraud_Detection.mlflow/#/experiments/0/runs/4cf4e215286b499e8d68777603c0d5e5
🧪 View experiment at: https://dagshub.com/electrolizzys/Fraud_Detection.mlflow/#/experiments/0
